In [116]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os

In [117]:


def read_input( train_path,test_path ):
    train = pd.read_csv(train_path)
    #print(train.head())
    print('df.shape :',train.shape)
    test = pd.read_csv(test_path)
    #print(test.head())
    print('df.shape :',test.shape)
    return train,test

def spilt_data( train,test ):
    value_columns = []
    for i in range(1,50):
        string = 'value_'+str(i)
        value_columns.append(string)   

    train_y = train[value_columns]
    test_y = test[value_columns]

    for col in value_columns:
        train.pop(col)
        test.pop(col)

    train_x = train
    test_x = test
    
    return train_y,test_y,train_x,test_x

In [118]:
def output(outputfile,predlist):
    np.savetxt(outputfile, predlist, delimiter=",")
    
def getscore(test_y,predict_test):
    #compute Accuracy
    tmp = abs( test_y - predict_test )/abs( test_y )
    #print(tmp)
    #print(type(tmp))
    #print("-----------------------------------------------------------")
    tmp[tmp >1] =1
    #print(tmp**2)
    square = tmp**2
    #print(square.shape[1])
    squ_sum = square.sum()
    #print(sum1)
    n = square.shape[0] * square.shape[1]
    #print(n)
    score = 100 - 100 * (np.sqrt(squ_sum/n))
    print("accuracy:" + str(score))
    
    return score,squ_sum

In [123]:
# xgboost模型


import xgboost as xgb
def xgb_train( train_x,train_y,test_x,test_y,testout_path ,test_file_name,gdb_type):
    xgb_len = len(test_y.T)
    maxdepth = [30]
    lr = [0.07]
    nest = [150]

    #train all case
    max_score = 0
    best_md = 0
    best_l = 0
    best_n = 0
    for md in maxdepth:
        for l in lr:
            for n in nest:
                predict_test = []
                print( "md:" + str(md) )
                print( "lr:" + str(l) )
                print( "nest:" + str(n) )

                for i in range(xgb_len):
                    #print("value" +  str(i) +"  training" )
                    params = {
                    "max_depth":int(md),
                    "learning_rate":float(l),
                    "n_estimators":int(n),
                    
                    #"subsample":0.9,
                    #"colsample_bytree":0.8
                    }

                    clf = xgb.XGBRegressor(**params)
                    clf.fit(train_x,train_y.T[i])
                    predict_test.append(clf.predict(test_x))

                predict_test = np.array(predict_test).T
                #print(predict_test)
                testoutput = testout_path + "\\"+ test_file_name +"_md_" + str(md)  + "lr_" + str(l)  + "nset_" + str(n) +'.csv' 
                errorout = testout_path + "\\"+ test_file_name +"md_" + str(md)  + "lr_" + str(l)  + "nset_" + str(n) +'_error.csv' 
                output(testoutput,(abs(predict_test)))
                output(errorout,(abs(predict_test - test_y)/abs( test_y ))**2)

                score,squ_sum = getscore(test_y,predict_test)
                """
                if (score > max_score):
                    max_score = score
                    best_md = md
                    best_l = l
                    best_n = n
                """

    print("max_score:" +str(max_score))
    print("best_md:" +str(best_md))
    print("best_l:" +str(best_l))
    print("best_n:" +str(best_n))
    return squ_sum

            
            
            

In [124]:

train_paths=[]
test_paths=[]
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1vm40c.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9v125c.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9vm40c.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_tt1p0v25c.csv')

train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ff0p99v125c_base_400.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ff0p99vm40c_base_400.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ss0p81v125c_base_400.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ss0p81vm40c_base_400.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_tt0p9v25c_base_400.csv')

test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1vm40c_beta_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9v125c_beta_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9vm40c_beta_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_tt1p0v25c_beta_100.csv')

test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ff0p99v125c_alpha_100.csv')
test_paths.append( r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ff0p99vm40c_alpha_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ss0p81v125c_alpha_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ss0p81vm40c_alpha_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_tt0p9v25c_alpha_100.csv')

for i in range(len(test_paths)):
    print(train_paths[i])
    print(test_paths[i])




C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1vm40c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1vm40c_beta_100.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9v125c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9v125c_beta_100.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9vm40c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9vm40c_beta_100.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_tt1p0v25c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_

In [125]:
squ_sum = 0.0
n = 0
for i in range(len(test_paths)):
    print( "train_file:" + train_paths[i] )
    print( "test_file:" + test_paths[i] )
    train,test = read_input( train_paths[i],test_paths[i] )
    train_y,test_y,train_x,test_x = spilt_data( train,test )
    
    n = n + test_y.shape[0] * test_y.shape[1]
    print("n:" + str(n))
    #dataframe -> numpy.array.value
    train_x = train_x.values
    train_y = train_y.values
    test_x = test_x.values
    test_y = test_y.values
    testout_path = r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test_record'
    file,filesplit = os.path.split(test_paths[i])
    test_file_name = filesplit.split('.')[0]
    #print(test_file_name)
    squ_sum = squ_sum+xgb_train( train_x,train_y,test_x,test_y,testout_path ,test_file_name)


score = 100 - 100 * (np.sqrt(squ_sum/n))
print("total_score:" + str(score))
    
    
    

train_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv
test_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv
df.shape : (18858, 74)
df.shape : (5658, 74)
n:277242
md:30
lr:0.07
nest:150


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  


accuracy:60.25000648520099
max_score:0
best_md:0
best_l:0
best_n:0
train_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1vm40c.csv
test_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1vm40c_beta_100.csv
df.shape : (18858, 74)
df.shape : (5658, 74)
n:554484
md:30
lr:0.07
nest:150
accuracy:59.293276417709805
max_score:0
best_md:0
best_l:0
best_n:0
train_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9v125c.csv
test_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9v125c_beta_100.csv
df.shape : (18858, 74)
df.shape : (5658, 74)
n:831726
md:30
lr:0.07
nest:150
accuracy:57.49720577017783
max_score:0
best_md:0
best_l:0
best_n:0
train_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9vm40c.csv
test_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost